## 0. 골드 Gold 레이어의 개념

Oracle AI Data Platform의 메달리온 구조에서 골드 레이어는 실버 단계에서 정제된 데이터를 기반으로 
실제 비즈니스에 바로 사용할 수 있는 최종 분석 결과와 예측 결과를 저장하는 단계입니다.

브론즈 레이어는 원본 데이터 보관,  
실버 레이어는 모델 학습과 분석이 가능한 표준 데이터셋 준비에 목적이 있다면,  
골드 레이어는 이 데이터를 바탕으로 생성된 결과 값을 업무 시스템이나 대시보드에서 바로 활용할 수 있도록 제공하는 역할을 합니다.

---

## 골드 레이어의 핵심 목적

- 모델이 예측한 결과 값을 저장해 비즈니스에서 바로 활용  
- 고객별 점수, 분류 결과, 확률 값 등 의미 있는 정보 제공  
- 분석가나 운영 시스템에서 조회하거나 후처리에 활용하도록 안정적으로 저장  
- 반복 예측 수행 시 누적 데이터로 활용할 수 있는 저장 구조 구성  

---

## 실버 → 골드 단계가 필요한 이유

| 실버 데이터 특징                       | 골드 레이어에서 수행하는 작업                    |
| -------------------------------------- | ----------------------------------------------- |
| 모델 입력을 위한 정제된 상태            | 예측된 결과를 생성 및 저장                      |
| 범주형 인코딩, 피처 벡터 등 기술적 컬럼 | 비즈니스에 필요한 컬럼만 남기고 정리           |
| 학습용 데이터 구조 중심                 | 운영 환경에서 바로 조회 가능한 형태로 전환      |
| 분석 또는 모델링 목적 중심              | 고객 단위 결과를 저장해 서비스 시스템과 연동    |

실버 레이어는 모델을 위한 데이터 준비 단계이고 골드 레이어는 예측 결과를 비즈니스가 실제로 활용할 수 있도록 제공하는 단계입니다.

---

## 0-1. 이 노트북에서 수행하는 전체 흐름

이 노트북에서는 실버 레이어에서 생성된 고객 데이터와 이전에 학습해 저장해둔 모델을 기반으로 
새로운 예측 값을 생성하고 해당 예측 결과를 골드 레이어에 저장합니다.

1. 골드 작업에 필요한 파라미터 설정  
2. 실버 레이어에서 현재 고객 데이터를 불러오기  
3. 저장된 머신러닝 모델 로드  
4. 고객 데이터에 대한 이탈 예측 실행  
5. 비즈니스에서 필요한 컬럼만 선택해 골드 저장용 구조 생성  
6. 골드 레이어 스키마와 테이블 준비  
7. 골드 테이블 최초 생성 후 예측 결과 적재  
8. 이후 실행에서는 예측 결과를 테이블에 누적 저장  

이 과정을 통해 골드 레이어에 고객별 최신 이탈 예측 결과를 지속적으로 쌓을 수 있습니다.


## 1. 파라미터 설정

In [1]:
# Define parameters (can set parameters in a workflow job)
silver_catalog="demo_telco_churn_silver"
silver_schema="telco_churn_historical"

## 2. 실버 레이어에서 현재 고객 데이터 로드

현재 시점의 고객 기반 데이터를 로드합니다.

이 데이터에는 고객 정보, 과금 정보, 서비스 사용 정보 등이 포함되어 있으며, 골드 레이어에서 예측 결과를 생성하기 위한 입력 데이터로 사용됩니다.

아래 코드는 실버 카탈로그에서 데이터를 읽어오고 필요한 컬럼 정제 작업을 수행합니다.

In [1]:
# -----------------------------
# Load Data
# -----------------------------

data = spark.read.table(f"{silver_catalog}.{silver_schema}.telco_customer_base_par")

# 필터링 예시. totalcharges가 비어 있는 고객은 제거
data = data.filter(data.totalcharges.isNotNull())

# 스키마 확인
data.printSchema()


root
 |-- customerid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- seniorcitizen: integer (nullable = true)
 |-- partner: string (nullable = true)
 |-- dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- phoneservice: string (nullable = true)
 |-- multiplelines: string (nullable = true)
 |-- internetservice: string (nullable = true)
 |-- onlinesecurity: string (nullable = true)
 |-- onlinebackup: string (nullable = true)
 |-- deviceprotection: string (nullable = true)
 |-- techsupport: string (nullable = true)
 |-- streamingtv: string (nullable = true)
 |-- streamingmovies: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- paperlessbilling: string (nullable = true)
 |-- paymentmethod: string (nullable = true)
 |-- monthlycharges: double (nullable = true)
 |-- totalcharges: double (nullable = true)



## 3. 저장된 모델 불러오기

실버 레이어에서 학습해 저장해둔 모델을 불러옵니다.
이 모델에는 전처리 파이프라인과 알고리즘 설정이 모두 포함되어 있어 그대로 예측에 사용할 수 있습니다.

In [1]:
from pyspark.ml import PipelineModel
loaded_model = PipelineModel.load("/Workspace/demo_telco_churn_bronze_to_silver/telco_customer_churn_ml_model")

## 4. 고객 이탈 예측 실행

불러온 모델을 현재 고객 데이터에 적용하여 예측을 생성합니다.

예측 결과에는 churn 예측 값과 확률 값이 포함됩니다.

In [1]:
# -----------------------------
# Predictions
# -----------------------------

predictions = loaded_model.transform(data)

predictions.show()

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+--------------------+--------------------+--------------------+----------+
|customerid|gender|seniorcitizen|partner|dependents|tenure|phoneservice|multiplelines|internetservice|     onlinesecurity|       onlinebackup|   deviceprotection|        techsupport|        streamingtv|    streamingmovies|      contract|paperlessbilling|       paymentmethod|monthlycharges|totalcharges|            features|       rawPrediction|         probability|prediction|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+-------

## 5. 골드 레이어 저장용 데이터 생성

예측 결과에서 최종 골드 데이터로 저장할 컬럼만 선택합니다.

전처리 과정에서 생성된 기술적인 컬럼들은 제거하고 고객 식별자, 예측 값과 확률 값 등 비즈니스 분석에 필요한 컬럼만 남깁니다.

In [1]:
# Define parameters (can set parameters in a workflow job)
target_type   =oidlUtils.parameters.getParameter("TARGET_TYPE", "table")
target_format =oidlUtils.parameters.getParameter("TARGET_FORMAT", "adw")
gold_catalog      = "demo_telco_churn_gold"
gold_schema       = "idl"

In [1]:
# 예측 결과에서 필요한 컬럼만 선택

Input_columns = predictions.drop(
    "customerid_Index","gender_Index","partners_Index","dependents_Index",
    "phoneservice_Index","multiplelines_Index","internetservice_Index","onlinesecurity_Index",
    "onlinebackup_Index","deviceprotection_Index","techsupport_Index",
    "streamingtv_Index","streamingmovies_Index","contract_Index",
    "paperlessbilling_Index","paymentmethod_Index",
    "gender_OHE","partners_OHE","dependents_OHE","phoneservice_OHE",
    "multiplelines_OHE","internetservice_OHE","onlinesecurity_OHE","onlinebackup_OHE",
    "deviceprotection_OHE","techsupport_OHE","streamingtv_OHE","streamingmovies_OHE",
    "contract_OHE","paymentmethod_OHE","features","rawPrediction","probability"
)

probability 값이 벡터 형태라 노트북 예시에서는 임시 컬럼을 넣어 단일 값으로 표현하는 방식이 포함되어 있습니다. 

필요하면 아래 코드처럼 확률 단일 값을 생성할 수도 있습니다. 

In [1]:
from pyspark.sql.functions import lit

Input_columns = Input_columns.withColumn("probability", lit(60))

## 6. 골드 레이어 카탈로그 준비

골드 스키마가 존재하지 않으면 생성하고 테이블 목록을 확인하여 저장 준비 상태를 점검합니다.

In [1]:
gold_catalog = "demo_telco_churn_gold"
gold_schema  = "idl"

print(gold_catalog, gold_schema)

demo_telco_churn_gold idl


In [1]:
spark.sql(f"DROP TABLE IF EXISTS {gold_catalog}.{gold_schema}.telco_cust_churn_predictions")

DataFrame[]

## 7. 골드 테이블 최초 생성 및 적재

최초 한 번은 테이블을 생성하고 데이터를 저장합니다.

In [1]:
(
  Input_columns.write
  .mode("overwrite")
  .format("delta")
  .saveAsTable(f"{gold_catalog}.{gold_schema}.telco_cust_churn_predictions")
)

## 8. 이후 실행에서는 데이터 추가 저장(insert)

같은 테이블에 데이터를 누적해야 하는 경우 insertInto 함수를 사용합니다.

In [1]:
spark.sql(f"SELECT * FROM {gold_catalog}.{gold_schema}.telco_cust_churn_predictions LIMIT 10").show(truncate=False)

+----------+------+-------------+-------+----------+-------------+------------+-------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+-----------------------+--------------+--------------+------------+-----------+
|customerid|gender|seniorcitizen|partner|dependents|tenure       |phoneservice|multiplelines|internetservice|onlinesecurity     |onlinebackup       |deviceprotection   |techsupport        |streamingtv        |streamingmovies    |contract      |paperlessbilling|paymentmethod          |monthlycharges|totalcharges  |prediction  |probability|
+----------+------+-------------+-------+----------+-------------+------------+-------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+-----------------------+--------------+--------------+---